# Example Gradio Application for Car Price Prediction

- Set up dropdown and slider options for application
- Download one-hot encoder and scalers
- Create function for prediction 

In [3]:
!pip install gradio
!git clone https://github.com/biodatlab/deep-learning-skooldio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 K

In [4]:
import numpy as np
import pandas as pd
import gradio as gr
import joblib
import os.path as op

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [6]:
CAT_COLUMNS = ["Brand", "Model", "Engine", "Segment", "Province", "Color"]
brands = [
  'Toyota', 'Honda', 'Mazda', 'Mitsubishi',
  'Nissan', 'Suzuki'
]
models = [
  'Vios', 'Altis', 'Civic', 'Mazda3', 'Camry',
  'Mirage', 'Brio', 'Lancer Ex', 'Jazz', 'Accord',
  'Lancer', 'Yaris', 'Almera', 'City', 'Swift', 'Mazda2',
  'Teana', 'Note', 'Celerio', 'March', 'Tiida', 'Prius',
  'Ciaz', 'Sylphy', 'Pulsar', 'Attrage', 'Sunny'
]
engines = [
  1.5, 1.8, 1.7, 2.0, 1.2, 1.6, 2.4,
  2.5, 1.0, 1.3, 2.3, 3.0, 2.2
]
segments = ['B-Segment', 'C-Segment', 'D-Segment', 'Eco Car']
provinces = [
  'สงขลา', 'กรุงเทพมหานคร', 'สระบุรี', 'ชัยนาท', 'ระยอง', 'นครสวรรค์',
  'นนทบุรี', 'ตาก', 'สมุทรสาคร', 'เชียงใหม่', 'ลำปาง', 'สุพรรณบุรี', 'เชียงราย',
  'เพชรบุรี', 'พิษณุโลก', 'นครปฐม', 'อุดรธานี', 'สมุทรปราการ', 'ปทุมธานี',
  'นครราชสีมา', 'ชลบุรี', 'ปัตตานี', 'ราชบุรี', 'ลำพูน', 'กระบี่', 'ฉะเชิงเทรา',
  'พัทลุง', 'อ่างทอง', 'ขอนแก่น', 'ปราจีนบุรี', 'สุราษฎร์ธานี', 'ภูเก็ต',
  'หนองบัวลำภู', 'พิจิตร', 'พะเยา', 'ตราด', 'นครศรีธรรมราช', 'บุรีรัมย์',
  'ลพบุรี', 'อุตรดิตถ์', 'ยโสธร', 'อุบลราชธานี', 'สิงห์บุรี', 'พระนครศรีอยุธยา',
  'กาฬสินธุ์', 'สกลนคร', 'ร้อยเอ็ด', 'ระนอง', 'นครพนม', 'อุทัยธานี', 'จันทบุรี',
  'มหาสารคาม', 'กาญจนบุรี', 'แพร่', 'บึงกาฬ', 'กำแพงเพชร', 'สมุทรสงคราม',
  'สุโขทัย', 'ตรัง', 'แม่ฮ่องสอน', 'อำนาจเจริญ', 'นครนายก', 'ชัยภูมิ', 'พังงา',
  'สระแก้ว', 'สุรินทร์', 'นราธิวาส', 'สตูล', 'ประจวบคีรีขันธ์', 'เพชรบูรณ์', 'ศรีสะเกษ',
  'หนองคาย', 'ยะลา', 'น่าน'
]
colors = ['Gray', 'Black', 'Gold', 'Silver', 'Brown', 'White',
          'Red', 'Yellow', 'Blue', 'Green', 'Cyan', 'Orange']

examples = [
  ['Honda', 'Civic', 1.8, 'C-Segment', 'ตรัง', 'Gray', 2009, 185477.0],
  ['Honda', 'Accord', 2.4, 'D-Segment', 'ขอนแก่น', 'Black', 2003, 166508.0],
  ['Honda', 'Jazz', 1.5, 'B-Segment', 'กรุงเทพมหานคร', 'White', 2011, 62000.0],
  ['Honda', 'Civic', 1.8, 'C-Segment', 'พระนครศรีอยุธยา', 'White', 2012, 165346.0],
  ['Suzuki', 'Swift', 1.2, 'Eco Car', 'กรุงเทพมหานคร', 'White', 2016, 193000.0],
  ['Honda', 'City', 1.0, 'B-Segment', 'กรุงเทพมหานคร', 'Gray', 2020, 29000.0],
  ['Honda', 'City', 1.5, 'B-Segment', 'พิษณุโลก', 'Gray', 2007, 126208.0],
  ['Toyota', 'Yaris', 1.5, 'Eco Car', 'เชียงใหม่', 'White', 2013, 100000.0],
  ['Toyota', 'Altis', 1.6, 'C-Segment', 'กรุงเทพมหานคร', 'Silver', 2009, 260000.0],
  ['Honda', 'Civic', 1.8, 'C-Segment', 'กรุงเทพมหานคร', 'Silver', 2006, 232433.0],
]

In [7]:
class CarPriceDataset(Dataset):
    def __init__(self, X, y = None):
        self.X = X
        if y is not None:
            self.y = y
        else:
            self.y = None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

class CarPriceTwoLayerModel(nn.Module):
    def __init__(self, input_size, output_size, intermediate_dim = 10):
        super().__init__()
        # TODOs: Create two layers neural network
        self.linear1 = nn.Linear(input_size, intermediate_dim)
        self.linear2 = nn.Linear(intermediate_dim, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

def predict(model, data_loader):
    model.eval()
    y_pred_list = []
    for x in data_loader:
        y_pred = model(x.float())
        prediction = y_pred.detach().numpy()
        y_pred_list.extend(prediction)
    y_pred_list = np.concatenate(y_pred_list)
    return y_pred_list

In [15]:
base_path = "deep-learning-skooldio/saved_model/carprice/"
one_hot_encoder_path = op.join(base_path, "one_hot_encoder.joblib")
year_scaler_path = op.join(base_path, "year_scaler.joblib")
mileage_scaler_path = op.join(base_path, "mileage_scaler.joblib")
price_scaler_path = op.join(base_path, "price_scaler.joblib")

ohe = joblib.load(one_hot_encoder_path)
year_scaler = joblib.load(year_scaler_path)
mileage_scaler = joblib.load(mileage_scaler_path)
price_scaler = joblib.load(price_scaler_path)

MODEL = CarPriceTwoLayerModel(138, 1)
MODEL.load_state_dict(torch.load(op.join(base_path, "carprice_two_layer_model_mse_00015.pth")))

<All keys matched successfully>

# Create Gradio application

- Predict function
- Interface: Inputs (Dropdown and Slider), Output (Text)

In [24]:
def predict_car_price(
    brand: str, model: str, engine: float, segment: str, province: str,
    color: str, year: float, mileage: float
):
  df = pd.DataFrame([{
    "Brand": brand,
    "Model": model,
    "Engine": engine,
    "Segment": segment,
    "Province": province,
    "Color": color,
    "Year": year,
    "Mileage": mileage,
  }])
  features = np.hstack([
      ohe.transform(df[CAT_COLUMNS]),
      year_scaler.transform(df[["Year"]]),
      mileage_scaler.transform(df[["Mileage"]])
  ])
  feat_dataset = CarPriceDataset(features)
  dataloaders = DataLoader(feat_dataset, batch_size=32, shuffle=False)
  y_pred_lr = predict(MODEL, dataloaders)
  return int(price_scaler.inverse_transform(y_pred_lr.reshape(-1, 1)).ravel()[0])

In [27]:
interface = gr.Interface(
    fn=predict_car_price,
    inputs=[
      gr.Dropdown(brands, label="Brand", info="Select Car Brand"),
      gr.Dropdown(models, label="Model", info="Select Car Model"),
      gr.Dropdown(engines, label="Engine Size", info="Select Engine Size"),
      gr.Dropdown(segments, label="Car segment", info="Select Car Segment"),
      gr.Dropdown(provinces, label="Province", info="Select Province"),
      gr.Dropdown(colors, label="Color", info="Select Color"),
      gr.Slider(1990, 2023, label="Year", info="Select Year"),
      gr.Slider(0, 400000, label="Mileage", info="Select Mileage"),
    ],
    outputs=gr.Textbox(label="ราคาทำนาย (บาท)", placeholder="xxx,xxx (บาท)"),
    examples=examples,
    title="ทำนายราคารถมือสอง",
    description="ตัวอย่างแอพพลิเคชั่นสำหรับคำนวณราคารถมือสอง",
)
interface.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>